In [1]:
# # Run this and then restart the kernel at the start of each session to install
# # 'teotil3' in development mode
# !pip install -e /home/jovyan/projects/teotil3/

In [2]:
import os
import warnings

import nivapy3 as nivapy
import pandas as pd
import teotil3 as teo
from tqdm.notebook import tqdm

warnings.simplefilter(action="ignore", category=FutureWarning)

# Improving suspended sediment coefficients in TEOTIL3

## Notebook 04: Run TEOTIL

Updated coefficients from Notebook 03 have been pushed to the TEOTIL3 repository ([here](https://github.com/NIVANorge/teotil3/blob/main/data/spatially_static_background_coefficients.csv)) and the code updated to use concentration-based coefficients for "background" SS ([here](https://github.com/NIVANorge/teotil3/commit/0240db3a62f1ce6931e7eb5e574288c37c27b434)).

This notebook runs TEOTIL3 with the new coefficients so that changes to the model output can be evaluated.

## 1. Generate input files

In [3]:
eng = nivapy.da.connect_postgis()

Connection successful.


In [4]:
# User options
nve_data_year = 2024
st_yr, end_yr = 2013, 2023

# Parent folder in which to save model input files.
# A subfolder will be created in this folder based on the agricultural loss
# model chosen i.e. f"agri_{agri_loss_model}_loss"
input_data_csv_fold = r"/home/jovyan/shared/common/teotil3/annual_input_data/test_new_ss_coeffs"

# Folder for results
result_csv_fold = r"/home/jovyan/shared/common/teotil3/evaluation/test_new_ss_coeffs"

# Model options
nan_to_vass = True
add_offshore = True
order_coastal = False  # Recommend False, except for comparison with TEOTIL2
land_to_vass = True  # Recommend True, except for comparison with TEOTIL2
agri_loss_model = "annual"  # 'annual' or 'risk'

# Set output folder based on user choices
input_data_csv_fold = os.path.join(input_data_csv_fold, f"agri_{agri_loss_model}_loss")

In [5]:
%%time

# Generate model input files and save to disk
if not os.path.exists(input_data_csv_fold):
    os.makedirs(input_data_csv_fold)

years = range(st_yr, end_yr + 1)
for year in tqdm(years):
    gdf = teo.io.make_input_file(
        year,
        nve_data_year,
        eng,
        out_csv_fold=input_data_csv_fold,
        nan_to_vass=nan_to_vass,
        add_offshore=add_offshore,
        order_coastal=order_coastal,
        land_to_vass=land_to_vass,
        agri_loss_model=agri_loss_model,
    )

  0%|          | 0/11 [00:00<?, ?it/s]

100.00 % of regines assigned.
100.00 % of regines assigned.
100.00 % of regines assigned.
100.00 % of regines assigned.
100.00 % of regines assigned.
100.00 % of regines assigned.
100.00 % of regines assigned.
100.00 % of regines assigned.
100.00 % of regines assigned.
100.00 % of regines assigned.
100.00 % of regines assigned.
CPU times: user 4min 28s, sys: 6.64 s, total: 4min 34s
Wall time: 6min 38s


## 2. Run model

In [6]:
# Run TEOTIL3
years = range(st_yr, end_yr + 1)
df_list = []
for year in tqdm(years):
    csv_path = os.path.join(
        input_data_csv_fold,
        f"teotil3_input_data_nve{nve_data_year}_{year}.csv",
    )
    g = teo.model.run_model(
        csv_path,
        id_col="regine",
        next_down_col="regine_down",
        totals_from_subfracs=True,
    )
    res_df = teo.model.model_to_dataframe(
        g, id_col="regine", next_down_col="regine_down"
    )
    res_df["year"] = year
    df_list.append(res_df)
    
print("Merging...")
mod_df = pd.concat(df_list, axis="rows")

mod_csv = os.path.join(
    result_csv_fold,
    f"teo3_results_nve{nve_data_year}_{st_yr}-{end_yr}_agri-{agri_loss_model}-loss.csv",
)
mod_df.to_csv(mod_csv, index=False)

print("Done.")

  0%|          | 0/11 [00:00<?, ?it/s]

Merging...
Done.
